In [2]:
import pandas as pd
import numpy as np
from utils.find_root import find_project_root
import os

In [3]:
# Retrieve the project root dynamically and set it as working directory
project_root = find_project_root()
os.chdir(project_root)

In [4]:
weather_data = pd.read_csv('outputs/modelling/predictions/df_train_infer.csv')

In [5]:
weather_data.head()

,timestamp,temperature_2m,surface_pressure,wind_speed_10m,precipitation,temperature_2m_z,surface_pressure_z,wind_r,precip_log,precip_z_12h,...,hour_cos,month,month_sin,month_cos,if_score,is_if_anomaly,used_in_lstm_training,lstm_score,is_lstm_anomaly,anomaly_label
0,2017-03-02 23:00:00,5.2,1008.1,12.6,0.0,-0.368721,-0.129419,-0.073710,0.0,0.0,...,0.965926,3,1.0,6.123234e-17,0.284430,False,False,0.438835,0.0,Normal
1,2017-03-03 00:00:00,5.1,1007.6,12.4,0.0,-0.397814,-0.169181,-0.173770,0.0,0.0,...,1.000000,3,1.0,6.123234e-17,0.285126,False,False,0.411973,0.0,Normal
2,2017-03-03 01:00:00,5.6,1007.0,12.7,0.0,-0.250480,-0.216915,-0.216393,0.0,0.0,...,0.965926,3,1.0,6.123234e-17,0.281436,False,False,0.421142,0.0,Normal
3,2017-03-03 02:00:00,5.8,1006.3,12.3,0.0,-0.191445,-0.272623,-0.226229,0.0,0.0,...,0.866025,3,1.0,6.123234e-17,0.278497,False,False,0.438329,0.0,Normal
4,2017-03-03 03:00:00,5.6,1005.0,10.5,0.0,-0.250199,-0.376226,-0.288525,0.0,0.0,...,0.707107,3,1.0,6.123234e-17,0.278049,False,False,0.478197,0.0,Normal


In [4]:
feature_columns = ['temperature_2m','surface_pressure','wind_speed_10m','precipitation']
X = weather_data[feature_columns]
y_lstm = weather_data['lstm_score']
y_if = weather_data['if_score']

In [5]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y_lstm ,test_size=0.2, random_state=42)

In [7]:
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [8]:
weather_data['timestamp'] = pd.to_datetime(weather_data['timestamp'])
weather_data['date'] = weather_data['timestamp'].dt.date

In [9]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69409 entries, 0 to 69408
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   timestamp              69409 non-null  datetime64[ns]
 1   temperature_2m         69409 non-null  float64       
 2   surface_pressure       69409 non-null  float64       
 3   wind_speed_10m         69409 non-null  float64       
 4   precipitation          69409 non-null  float64       
 5   temperature_2m_z       69409 non-null  float64       
 6   surface_pressure_z     69409 non-null  float64       
 7   wind_r                 69409 non-null  float64       
 8   precip_log             69409 non-null  float64       
 9   precip_z_12h           69409 non-null  float64       
 10  precip_z_24h           69409 non-null  float64       
 11  hour                   69409 non-null  int64         
 12  hour_sin               69409 non-null  float64       
 13  h

In [10]:
threshold_lstm= np.percentile(weather_data['lstm_score'], 95)
print(threshold_lstm)

0.6425390051638658


In [12]:
!pip install lime
import lime
from lime.lime_tabular import LimeTabularExplainer


explainer = LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=feature_columns,
    mode='regression'

)
for date, day_group in weather_data.groupby('date'):
    lstm_high_rows = weather_data[weather_data['lstm_score'] >= 0.64].head(1000)


    for idx, row in lstm_high_rows.iterrows():
        row_input = row[feature_columns].values


        explanation = explainer.explain_instance(
            data_row=row_input,
            predict_fn=model.predict,
            num_features=4,
            num_samples = 200
        )

        print(f"Date: {row['timestamp']}, Score: {row['lstm_score']:.2f}")

        explanation.show_in_notebook()


lstm_lime_weights = {f.split()[0]: weight for f, weight in explanation.as_list()}

for feature in feature_columns:
            col_name = f'lstm_lime_{feature}'
            weather_data.at[idx, col_name] = lstm_lime_weights.get(feature, 0.0)


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

KeyboardInterrupt: 

In [ ]:
weather_data['lstm_lime_temperature_2m'].describe()


In [ ]:
weather_data['lstm_lime_temperature_2m'].value_counts()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y_if ,test_size=0.2, random_state=42)


model2 = RandomForestRegressor(n_estimators=200, random_state=42)
model2.fit(X_train1, y_train1)

In [ ]:
threshold_if= np.percentile(weather_data['if_score'], 3)
print(threshold_if)

In [ ]:
explainer = LimeTabularExplainer(
    training_data=X_train1.values,
    feature_names=feature_columns,
    mode='regression'
)

lime_explanations = []

for date, day_group in weather_data.groupby('date'):
    low_if_rows = day_group[day_group['if_score'] <= 0.033].head(1000)

    for idx, row in low_if_rows.iterrows():
        row_input1 = row[feature_columns].values


        explanation2 = explainer.explain_instance(
            data_row=row_input1,
            predict_fn=model2.predict,
            num_features=4
            num_samples=200
        )
        print(f"Date: {row['timestamp']}, Score: {row['lstm_score']:.2f}")

        explanation.show_in_notebook()


if_lime_weights = {f.split()[0]: weight for f, weight in explanation2.as_list()}


for feature in feature_columns:
            col_name = f'IF_lime_{feature}'
            weather_data.at[idx, col_name] = if_lime_weights.get(feature, 0.0)

